### LLM Tools 
#### Airline assistance, an example

In [3]:
import os
from dotenv import load_dotenv
import json
from openai import OpenAI
import gradio as gr

e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"openai api key exists and starts with {openai_api_key[:2]}")
else:
    print("no key found")
openai = OpenAI()
MODEL= "gpt-4.1-mini"


openai api key exists and starts with sk


In [ ]:
system_message = """You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""
def chat(message, history):
    print(history)
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


[]
[{'role': 'user', 'metadata': None, 'content': [{'text': 'hello', 'type': 'text'}], 'options': None}, {'role': 'assistant', 'metadata': None, 'content': [{'text': 'Hello! How can I assist you with your FlightAI needs today?', 'type': 'text'}], 'options': None}]


In [ ]:
dest_ticket_price ={"london" : "499$", "paris" : "899$", "Benagluru" : "988$", "tokiyo" : "455$"}

def get_ticket_price(destination):
    destination = destination.lower()
    print(f"looking for destination: {destination}")
    price = dest_ticket_price.get(destination, "price not found")
    return price


looking for destination: london


'499$'

In [18]:
price_function={
    "name" : "get_ticket_price",
    "description" : "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [20]:
tools = [{"type" : "function", "function" : price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [56]:
def airline_assistant_chat(message,history):
    history = [{"role" : h["role"], "content" : h["content"]} for h in history]
    messages = [{"role" : "system", "content" : system_message}] + history + [{"role" : "user", "content" : message}]
    ## here you are calling llm with the tools that you have
    response = openai.chat.completions.create(model = MODEL, messages = messages, tools = tools)
    print(f"llm response : {response.choices[0]}")
    if response.choices[0].finish_reason =="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        tools_response = handle_tools_call(message)
        messages.append(tools_response)
        print(f"calling llm with messages: {messages}")
        response = openai.chat.completions.create(model = MODEL, message = messages)
        
    return response.choices[0].message.content

In [51]:
def handle_tools_call(message):
    tool_calls = message.tool_calls[0]
    fun_name = tool_calls.function.name
    if fun_name == "get_ticket_price":
        arguments = json.loads(tool_calls.function.arguments)
        city = arguments.get('destination')
        price = get_ticket_price(city)
        print(f"found price : {price}")
        response = {
            "role": "tool",
            "content": price,
            "tool_call_id": message.tool_calls[0].id
        }
    return response

In [57]:
gr.ChatInterface(fn=airline_assistant_chat).launch()

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


llm response : Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_MWJh6cZwqs3PtwURFk9TARKX', function=Function(arguments='{"destination":"London"}', name='get_ticket_price'), type='function')]))
looking for destination: London
found price : price not found
calling llm with messages: [{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlightAI.\nGive short, courteous answers, no more than 1 sentence.\nAlways be accurate. If you don't know the answer, say so.\n"}, {'role': 'user', 'content': 'what is the cost to fly to london'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_MWJh6cZwqs3PtwURFk9TARKX', function=Function(arguments='{"destination":

Traceback (most recent call last):
  File "e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\gradio\blocks.py", line 2125, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\gradio\blocks.py", line 1605, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\gradio\utils.py", line 1033, in async_w